In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

In [ ]:
api_url = "https://api.binance.com/api/v3/klines"

symbol = 'BNBUSDT'
interval = '1m'

params = {
    'symbol': symbol,
    'interval': interval,
}

In [ ]:
response = requests.get(api_url, params=params)
klines = response.json()

In [ ]:
df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])

# Convert timestamps to datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Keep only relevant columns
df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]


In [ ]:
def calculate_moving_averages(df, window_sizes):
    for window_size in window_sizes:
        column_name = f'MA_{window_size}'
        df[column_name] = df['close'].rolling(window=window_size).mean()
    return df

In [ ]:
window_sizes = [7, 25, 99]
df = calculate_moving_averages(df, window_sizes)

In [ ]:
df.to_csv('bnb_price_data.csv', index=False)

In [ ]:
################# Model Time ####################
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

In [ ]:
df = pd.read_csv('bnb_price_data.csv')

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp']) 
df['next_min_close'] = df['close'].shift(-1) 
df['next_10min_close'] = df['close'].shift(-10) 

In [ ]:
features = df[['timestamp', 'MA_7', 'MA_25', 'MA_99',  'open', 'high', 'low', 'close', 'volume', 'next_min_close']]  

df_1m = features

features = features.drop(columns=['timestamp'])

# Drop rows with missing values
features = features.dropna()

# Target variable
target = features['next_min_close']

# Drop target variable from features
features = features.drop(columns=['next_min_close'])
scaler = StandardScaler()
features_standardized = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_standardized, target, test_size=0.2, random_state=42)

model_1m = LinearRegression()
model_1m.fit(X_train, y_train)

predictions = model_1m.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')
mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape:.2f}%')

In [ ]:
features_10m = df[['timestamp', 'MA_7', 'MA_25', 'MA_99',  'open', 'high', 'low', 'close', 'volume', 'next_10min_close']]  

df_10m = features_10m

features_10m = features_10m.drop(columns=['timestamp'])

# Drop rows with missing values
features_10m = features_10m.dropna()

# Target variable
target_10m = features_10m['next_10min_close']

# Drop target variable from features
features_10m = features_10m.drop(columns=['next_10min_close'])
scaler_10m = StandardScaler()
features_standardized_10m = scaler_10m.fit_transform(features_10m)

X_train_10m, X_test_10m, y_train_10m, y_test_10m = train_test_split(features_standardized_10m, target_10m, test_size=0.2, random_state=42)

model_10m = RandomForestRegressor(n_estimators=100, random_state=42)
model_10m.fit(X_train_10m, y_train_10m)

predictions_10m = model_10m.predict(X_test_10m)
mae_10m = mean_absolute_error(y_test_10m, predictions_10m)
print(f'Mean Absolute Error: {mae_10m}')
mape_10m = np.mean(np.abs((y_test_10m - predictions_10m) / y_test_10m)) * 100
print(f'Mean Absolute Percentage Error (MAPE): {mape_10m:.2f}%')

In [ ]:
predictions_1m = model_1m.predict(features_standardized)
df_singals = df_1m.dropna()
print(df_singals)
# Create a DataFrame to store signals
df_singals['predicted_price'] = predictions_1m

df_singals['predicted_price'] = df_singals['predicted_price'].shift(1)

In [ ]:
predictions_10m = model_10m.predict(features_standardized_10m)
df_singals_10m  = df_10m.dropna()
# Create a DataFrame to store signals
df_singals_10m['predicted_price'] = predictions_10m

last_timestamp = df_singals_10m['timestamp'].max()
future_timestamps = pd.date_range(last_timestamp + timedelta(minutes=10), periods=10, freq='1T')

# Create a DataFrame with the future timestamps
future_df = pd.DataFrame({'timestamp': future_timestamps})

extended_df = pd.concat([df_singals_10m, future_df], ignore_index=True)

extended_df['predicted_price'] = extended_df['predicted_price'].shift(10)

#print(predictions_10m)
print(extended_df)

In [ ]:
candlestick_trace  = go.Candlestick(x=df['timestamp'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])


trace_ma_7 = go.Scatter(x=df['timestamp'], y=df['MA_7'], mode='lines', name='MA(7)', line=dict(color='green', dash='dash'))
trace_ma_25 = go.Scatter(x=df['timestamp'], y=df['MA_25'], mode='lines', name='MA(25)', line=dict(color='red', dash='dash'))
trace_ma_99 = go.Scatter(x=df['timestamp'], y=df['MA_99'], mode='lines', name='MA(99)', line=dict(color='purple', dash='dash'))

pred_1m = go.Scatter(x=df_singals['timestamp'], y=df_singals['predicted_price'], mode='lines', name='predicted-price', line=dict(color='blue'))

pred_10m = go.Scatter(x=extended_df['timestamp'], y=extended_df['predicted_price'], mode='lines', name='predicted-price-10m', line=dict(color='orange'))


fig = go.Figure(data=[candlestick_trace, trace_ma_7, trace_ma_25, trace_ma_99, pred_1m, pred_10m])

fig.update_layout(title=f'Historical Price Data for {symbol}',
                  xaxis_title='Timestamp',
                  yaxis_title='Price (USDT)',
                  xaxis_rangeslider_visible=False)

# Show the interactive plot in a browser
fig.show()